In [1]:
import os
from glob import glob
from zipfile import ZipFile
from natsort import natsorted
from Config import PERIG_CLASSES, MODERN_CLASSES_V2
from huggingface_hub import snapshot_download
from Source.Utils import create_dir, show_sample_pair
from Source.Trainer import MultiSegmentationTrainer

d:\Projects\Python\ImageSegmentation\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
layout_dataset = "BDRC/LayoutSegmentation_Dataset"

dataset_path = snapshot_download(
            repo_id=f"{layout_dataset}",
            repo_type="dataset",
            cache_dir="Datasets")

with ZipFile(f"{dataset_path}/data.zip", 'r') as zip:
    zip.extractall(f"{dataset_path}")

print(f"downloaded and extracted the dataset to: {dataset_path}")

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


In [2]:
dataset_path = "Datasets/WesternTiledDataset"

train_data = os.path.join(dataset_path, "train")
val_data = os.path.join(dataset_path, "val")
test_data = os.path.join(dataset_path, "test")

train_x = natsorted(glob(f"{train_data}/images/*.png"))
train_y = natsorted(glob(f"{train_data}/masks/*.png"))

valid_x = natsorted(glob(f"{val_data}/images/*.png"))
valid_y = natsorted(glob(f"{val_data}/masks/*.png"))

test_x = natsorted(glob(f"{test_data}/images/*.png"))
test_y = natsorted(glob(f"{test_data}/masks/*.png"))

print(f"Training data => Images: {len(train_x)}, Masks: {len(train_y)}")
print(f"Validation data => Images: {len(valid_x)}, Masks: {len(valid_y)}")
print(f"Test data => Images: {len(test_x)}, Masks: {len(test_y)}")

Training data => Images: 16914, Masks: 16914
Validation data => Images: 16126, Masks: 16126
Test data => Images: 16130, Masks: 16130


In [3]:
patch_size = 512
batch_size = 32

In [4]:
output_dir = os.path.join(dataset_path, "Output")
create_dir(output_dir)

segmentation_trainer = MultiSegmentationTrainer(
    train_x,
    train_y,
    valid_x,
    valid_y,
    test_x,
    test_y,
    image_width=patch_size,
    image_height=patch_size,
    batch_size=batch_size,
    network="deeplab",
    output_path=output_dir,
    classes=MODERN_CLASSES_V2)

Initializing Mutliclass Segmentation trainer...


e:\Projects\Python\ImageSegmentation\.venv\Lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: DiceScore metric currently defaults to `average=micro`, but will change to`average=macro` in the v1.9 release. If you've explicitly set this parameter, you can ignore this warning.
  warnings.warn(*args, **kwargs)


In [5]:
train_sample = next(iter(segmentation_trainer.train_ds))

In [ ]:
epochs = 12
segmentation_trainer.train(epochs=epochs)

In [ ]:
segmentation_trainer.export2onnx(segmentation_trainer.model, model_name="modernbookformat")